In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
import IPython.display as display
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.decomposition import FastICA
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from typing import Union
from sklearn.model_selection import ShuffleSplit
from sklearn.decomposition import FactorAnalysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import KernelPCA
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from sklearn.preprocessing import FunctionTransformer
from multiprocess import Pool # https://pypi.org/project/multiprocess/
import time_constants as tc
from helper_functions_dataloader import *

In [ ]:
#Read Lever Times
tc.updateTimes(-10, 10)

TIME_LOWER = tc.TIME["LOWER"]
TIME_UPPER = tc.TIME["UPPER"]
TOT_TIME = tc.TIME["TOTAL"]

# os.path.join() will be useful for switching between mac, windows, linux 

pal = sns.color_palette('husl', 9)
TIME_LOWER, TIME_UPPER, TOT_TIME

In [ ]:
mTypes = {'WT':['152', '153', '174', '175', '177', '180', '181'], 
          'D1':['D1#14', 'D1#15', 'D1#21', 'D1#7', 'D1#33'],
          'D2':['D2#13', 'D2#24', 'D2#36', 'D2#23', 'D2#38', 'D2#52']}

In [ ]:
# Assuming that this code is in the same directory as each folder, which is organized as:
'''
Sucrose_data_and_code(folder)
|- This code
|- Rodent(folder)
   |- rodent_day1_...
   |- rodent_day-whatever...csv etc.
|- Another_Rodent(folder)
   |- blah blah...csv
'''
days = ['Day11']
USE_CUT = False
days

In [ ]:
# Create data objects
mTypeDats = dict()
for key in mTypes:
    print(key)
    rodents = mTypes[key]
    mTypeDats[key] = []
    for d in days:
        data = dict()
        for rodent in rodents:
            print(rodent, d)
            dat = DataLoader("data\\" + rodent, rodent, d, use_manifold_transformed=True, use_cut=USE_CUT)
            data[rodent] = dat
        mTypeDats[key].append((d, data))
mTypeDats

In [ ]:
FRAC_PART = 5
colTimes = []
addr = float(TIME_LOWER)
while addr <= TIME_UPPER:
    colTimes.append(addr)
    addr += 1*FRAC_PART/10
    addr = np.round(addr, 1)
colNames = ["[{t1},{t2})".format(t1 = colTimes[i], t2 = colTimes[i+1]) for i in range(len(colTimes) - 1)]
print(colNames)
my_index = pd.MultiIndex(levels=[[],[],[],[],[]], codes=[[],[],[],[],[]], names=[u'Type', u'Day', u'Rodent', u'iter', u'nALP'])
df_results = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
df_results_shuff = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)

In [ ]:
def parallelRun(inp):
    from sklearn.model_selection import ShuffleSplit
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.pipeline import Pipeline
    from sklearn.svm import LinearSVC
    from sklearn.svm import SVC
    from sklearn.decomposition import FactorAnalysis
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import balanced_accuracy_score
    import numpy as np
    import pandas as pd
    import time_constants as tc
    from sklearn.preprocessing import FunctionTransformer
    from helper_functions_dataloader import expandTimeWindow
    
    iter,mTypeDats,TOT_TIME,TIME_LOWER,TIME_UPPER,FRAC_PART,IS_PCA,colNames,my_index = inp
    df_results = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
    df_results_shuff = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
    mTypeDFs = dict()
    aggFunc = lambda x,y: np.concatenate((np.nanmean(x, axis=0), np.nanmean(y, axis=0)), axis=0)
    tc.updateTimes(TIME_LOWER, TIME_UPPER)

    for key in mTypeDats:
        rodents = mTypeDats[key]
        mTypeDFs[key] = []
        for day, mice in rodents:
            for dlKey in mice:
                print(key, day, dlKey)
                # Load in data from dataloader
                loader = mice[dlKey]
                if loader.empty or len(loader.times_ALP) == 0:
                    continue
                LP1 = expandTimeWindow(loader.df_times, loader.times_ALP, time_grouped=True)
                LP2 = expandTimeWindow(loader.df_times, loader.times_ALP, shuffle=True, time_grouped=True)
                LP3 = expandTimeWindow(loader.df_times, loader.times_ALP, shuffle=True, time_grouped=True)
                LP4 = expandTimeWindow(loader.df_times, loader.times_ALP, shuffle=True, time_grouped=True)

                # Set up data split
                numModels = TOT_TIME//FRAC_PART if FRAC_PART != 1 else TOT_TIME-1
                skf = ShuffleSplit(n_splits=1, test_size=.4)
                agg = {i:[] for i in range(numModels)}
                aggShuff = {i:[] for i in range(numModels)}

                for train_index, test_index in skf.split(loader.times_ALP):
                    # Split data points
                    train_set_1, train_set_2 = LP1[train_index], LP2[train_index]
                    test_set_1, test_set_2 = LP1[test_index], LP2[test_index]

                    train_set_3, train_set_4 = LP3[train_index], LP4[train_index]
                    test_set_3, test_set_4 = LP3[test_index], LP4[test_index]

                    # Set up dim reduction pipeline
                    pipe = Pipeline([('scaler', StandardScaler()),('pca', PCA(n_components=3))])
                    pca_fit_inp = aggFunc(train_set_1,train_set_2)
                    pipe.fit(pca_fit_inp)

                    pipe_shuff = Pipeline([('scaler', StandardScaler()),('pca', PCA(n_components=3))])
                    pca_fit_inp_shuff = aggFunc(train_set_3,train_set_4)
                    pipe_shuff.fit(pca_fit_inp_shuff)

                    for timeRange in range(numModels):
                        # Set up time ranges and helper functions
                        lTrain, uTrain = timeRange*FRAC_PART, (timeRange+1)*FRAC_PART
                        remove_nan = lambda x: list(filter(lambda y: not np.any(np.isnan(y)), x))
                        reshape = lambda x: x.reshape(x.shape[0]*x.shape[1], x.shape[2])
                        def res_conc_trans(x,y,pipe):
                            l_arr = remove_nan(reshape(x[:,lTrain:uTrain,:]))
                            r_arr = remove_nan(reshape(y[:,lTrain:uTrain,:]))
                            res_conc = np.concatenate((l_arr,r_arr), axis=0)
                            if np.any(np.isnan(res_conc)):
                                raise Exception("bruh")
                            return pipe.transform(res_conc), len(l_arr), len(r_arr)
                            
                        # Prep test data
                        train_dat, train_ll, train_rl = res_conc_trans(train_set_1, train_set_2, pipe)
                        test_dat, test_ll, test_rl = res_conc_trans(test_set_1, test_set_2, pipe)
                        train_dat_shuff, train_ll_shuff, train_rl_shuff = res_conc_trans(train_set_3, train_set_4, pipe_shuff)
                        test_dat_shuff, test_ll_shuff, test_rl_shuff = res_conc_trans(test_set_3, test_set_4, pipe_shuff)
                    
                        train_labels = [0]*train_ll + [1]*train_rl
                        test_labels = [0]*test_ll + [1]*test_rl
                        train_labels_shuff = [0]*train_ll_shuff + [1]*train_rl_shuff
                        test_labels_shuff = [0]*test_ll_shuff + [1]*test_rl_shuff

                        # SVM and scoring
                        svm = Pipeline([('scaler', StandardScaler()),('svm', LinearSVC(C=0.9, max_iter=10000, dual=False))])
                        svm.fit(train_dat, train_labels)                                
                        pred_labels = svm.predict(test_dat)
                        accuracy = balanced_accuracy_score(test_labels, pred_labels, sample_weight=[test_rl/(test_ll + test_rl)]*test_ll + [test_ll/(test_ll + test_rl)]*test_rl)
                        agg[timeRange].append(accuracy)

                        # SVM and scoring of shuffle
                        svm_shuff = Pipeline([('scaler', StandardScaler()),('svm', LinearSVC(C=0.9, max_iter=10000, dual=False))])
                        svm_shuff.fit(train_dat_shuff, train_labels_shuff)                                
                        pred_labels = svm_shuff.predict(test_dat_shuff)
                        accuracy = balanced_accuracy_score(test_labels_shuff, pred_labels, sample_weight=[test_rl_shuff/(test_ll_shuff + test_rl_shuff)]*test_ll_shuff + [test_ll_shuff/(test_ll_shuff + test_rl_shuff)]*test_rl_shuff)
                        aggShuff[timeRange].append(accuracy)
                
                df_results.loc[(key, day, dlKey, iter, len(loader.times_ALP)),:] = [np.mean(agg[i]) for i in agg]
                df_results_shuff.loc[(key, day, dlKey, iter, len(loader.times_ALP)),:] = [np.mean(aggShuff[i]) for i in aggShuff]
    return df_results,df_results_shuff

In [ ]:
if __name__ ==  '__main__':
    num_iters = 100
    num_processors = 11
    p=Pool(processes = num_processors)
    arr = p.map(parallelRun,[(i,mTypeDats,TOT_TIME,TIME_LOWER,TIME_UPPER,FRAC_PART,IS_PCA,colNames,my_index) for i in range(0,num_iters)])
    results_arr, shuff_arr = [x for (x,_) in arr],[y for (_,y) in arr]
    df_results = pd.concat(results_arr)
    df_results_shuff = pd.concat(shuff_arr)

In [ ]:
for key in mTypeDats:
    rodents = mTypeDats[key]
    for day, mice in rodents:
        plt.plot(df_results.loc[key, day, :].mean(), label="ALP v Shuff")
        plt.plot(df_results_shuff.loc[key, day, :].mean(), label="Shuff v Shuff")
        plt.title(key + " " + day)      
        plt.ylim((0.375, 0.775))
        plt.xticks(rotation=90)
        plt.legend() 
        plt.show()
        plt.clf()

In [ ]:
df_results.to_csv('[3PC]sucrose-alp-shuff.csv')
df_results_shuff.to_csv('[3PC]sucrose-shuff-shuff.csv')

In [ ]:
my_index = pd.MultiIndex(levels=[[],[],[]], codes=[[],[],[]], names=[u'Type', u'Day', u'Rodent'])
df_result_means = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
df_result_sems = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
df_result_shuff_means = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
df_result_shuff_sems = pd.DataFrame([], columns=colNames, index=my_index, dtype=float)
for key in mTypeDats:
    rodents = mTypeDats[key]
    for day, mice in rodents:
        for m in mice:
            df_result_means.loc[(key, day, m),:] = df_results.loc[key, day, :].mean()
            df_result_sems.loc[(key, day, m),:] = np.sum([df_results.loc[key, day, x, :].sem()**2 for x in mTypes[key] if not df_results.loc[key, day, x, :].empty], axis=0)**0.5
            
            df_result_shuff_means.loc[(key, day, m),:] = df_results_shuff.loc[key, day, :].mean()
            df_result_shuff_sems.loc[(key, day, m),:] = np.sum([df_results_shuff.loc[key, day, x, :].sem()**2 for x in mTypes[key] if not df_results_shuff.loc[key, day, x, :].empty], axis=0)**0.5
        
df_result_sems

In [ ]:
df_result_means.to_csv('[3PC][per-rodent]sucrose-alp-shuff-mean.csv')
df_result_sems.to_csv('[3PC][per-rodent]sucrose-alp-shuff-sem.csv')
df_result_shuff_means.to_csv('[3PC][per-rodent]sucrose-shuff-shuff-mean.csv')
df_result_shuff_sems.to_csv('[3PC][per-rodent]sucrose-shuff-shuff-sem.csv')

In [ ]:
for key in mTypeDats:
    rodents = mTypeDats[key]
    for day, mice in rodents:
        for m in mice:
            plt.plot(df_results.loc[key, day, m,:].mean(), label="ALP v Shuff")
            plt.plot(df_results_shuff.loc[key, day, :].mean(), label="Shuff v Shuff")
            plt.title(m + " " + key + " " + day)      
            plt.ylim((0.375, 0.775))  
            plt.xticks(rotation=90)
            plt.legend() 
            plt.show()
            # plt.savefig(f"[-10,10][PCA-full][meanPCA][centerTimeNorm]ALPvBL\\"+m, bbox_inches='tight')
            plt.clf()